In [30]:
# !pip install transformers
# !pip install sentencepiece
# !pip install matplotlib
# !pip install pandas
# !pip install numpy
# !pip install torch
# !pip install wandb
# !pip install tashaphyne
# !pip install emojis

from read_write_qrcd import *
from assets import *
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import sentencepiece
from datasets import Dataset
import pandas as pd
from datasets import load_dataset
from datasets import load_dataset, load_metric
import random
from transformers import AutoTokenizer
import transformers
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
import torch
from transformers import default_data_collator
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
from data_preprocess import *

In [31]:
passage_question_objects = read_JSONL_file('datasets/qrcd_v1.1_dev.jsonl')

Loaded 109 records from datasets/qrcd_v1.1_dev.jsonl
pq_id: 9:60-61_316
pq_id: 2:178-179_400
pq_id: 2:190-194_400
pq_id: 2:243-245_400
pq_id: 4:74-76_400
pq_id: 4:80-84_400
pq_id: 4:88-91_400
pq_id: 5:32-34_400
pq_id: 8:38-40_400
pq_id: 9:1-6_400
pq_id: 9:12-16_400
pq_id: 9:28-29_400
pq_id: 9:36-37_400
pq_id: 9:122-123_400
pq_id: 17:31-35_400
pq_id: 25:63-77_400
pq_id: 33:25-27_400
pq_id: 33:60-62_400
pq_id: 47:1-6_400
pq_id: 49:9-10_400
pq_id: 2:34-39_257
pq_id: 7:19-25_257
pq_id: 20:115-123_257
pq_id: 2:83-86_313
pq_id: 2:178-179_313
pq_id: 2:228-230_313
pq_id: 2:234-237_313
pq_id: 3:145-148_313
pq_id: 3:169-175_313
pq_id: 4:36-40_313
pq_id: 4:122-126_313
pq_id: 5:12-13_313
pq_id: 5:82-86_313
pq_id: 7:54-56_313
pq_id: 9:90-96_313
pq_id: 9:100-102_313
pq_id: 9:117-121_313
pq_id: 17:22-30_313
pq_id: 18:29-31_313
pq_id: 22:30-37_313
pq_id: 29:61-69_313
pq_id: 33:28-31_313
pq_id: 39:9-10_313
pq_id: 39:32-37_313
pq_id: 41:33-36_313
pq_id: 46:15-18_313
pq_id: 53:31-41_313
pq_id: 2:124-129_

In [32]:
df_file = json_to_dataframe(passage_question_objects)

In [33]:
df_file.head()

,id,context,question,answers
0,9:60-61_316,انما الصدقات للفقراء والمساكين والعاملين عليها...,ما هي مصارف الزكاه ؟,{'text': ['للفقراء والمساكين والعاملين عليها و...
1,2:178-179_400,يا ايها الذين امنوا كتب عليكم القصاص في القتلي...,متي يحل الاسلام دم الشخص ؟,{'text': ['كتب عليكم القصاص في القتلي الحر بال...
2,2:190-194_400,وقاتلوا في سبيل الله الذين يقاتلونكم ولا تعتدو...,متي يحل الاسلام دم الشخص ؟,{'text': ['قاتلوا في سبيل الله الذين يقاتلونكم...
3,2:243-245_400,الم تر الي الذين خرجوا من ديارهم وهم الوف حذر ...,متي يحل الاسلام دم الشخص ؟,"{'text': ['قاتلوا في سبيل الله'], 'answer_star..."
4,4:74-76_400,فليقاتل في سبيل الله الذين يشرون الحياه الدنيا...,متي يحل الاسلام دم الشخص ؟,"{'text': ['الذين امنوا يقاتلون في سبيل الله', ..."


In [34]:
dataset = Dataset.from_pandas(df_file)

In [35]:
type(dataset)

datasets.arrow_dataset.Dataset

In [37]:
dataset[0]

{'id': '9:60-61_316',
 'context': 'انما الصدقات للفقراء والمساكين والعاملين عليها والمؤلفه قلوبهم وفي الرقاب والغارمين وفي سبيل الله وابن السبيل فريضه من الله والله عليم حكيم . ومنهم الذين يؤذون النبي ويقولون هو اذن قل اذن خير لكم يؤمن بالله ويؤمن للمؤمنين ورحمه للذين امنوا منكم والذين يؤذون رسول الله لهم عذاب اليم . ',
 'question': 'ما هي مصارف الزكاه ؟ ',
 'answers': {'answer_start': [13],
  'text': ['للفقراء والمساكين والعاملين عليها والمؤلفه قلوبهم وفي الرقاب والغارمين وفي سبيل الله وابن السبيل']}}

In [38]:
len(dataset)

109

In [39]:
# datasets = load_dataset("squad")
def visualize(datasets, n_questions=10):
    n = len(datasets)
    random_questions=random.choices(list(range(n)),k=n_questions)
    
    for i in random_questions:
        print(f"Context:{datasets[i]['context']}")
        print(f"Question:{datasets[i]['question']}")
        print(f"Answer:{datasets[i]['answers']['text']}")
        print(f"Answer Start in Text:{datasets[i]['answers']['answer_start']}")
        print("-"*100)
visualize(dataset)

Context:لقد تاب الله علي النبي والمهاجرين والانصار الذين اتبعوه في ساعه العسره من بعد ما كاد يزيغ قلوب فريق منهم ثم تاب عليهم انه بهم رءوف رحيم . وعلي الثلاثه الذين خلفوا حتي اذا ضاقت عليهم الارض بما رحبت وضاقت عليهم انفسهم وظنوا ان لا ملجا من الله الا اليه ثم تاب عليهم ليتوبوا ان الله هو التواب الرحيم . يا ايها الذين امنوا اتقوا الله وكونوا مع الصادقين . ما كان لاهل المدينه ومن حولهم من الاعراب ان يتخلفوا عن رسول الله ولا يرغبوا بانفسهم عن نفسه ذلك بانهم لا يصيبهم ظما ولا نصب ولا مخمصه في سبيل الله ولا يطئون موطئا يغيظ الكفار ولا ينالون من عدو نيلا الا كتب لهم به عمل صالح ان الله لا يضيع اجر المحسنين . ولا ينفقون نفقه صغيره ولا كبيره ولا يقطعون واديا الا كتب لهم ليجزيهم الله احسن ما كانوا يعملون . 
Question:ماذا يشمل الاحسان ؟ 
Answer:['ما كان لاهل المدينه ومن حولهم من الاعراب ان يتخلفوا عن رسول الله ولا يرغبوا بانفسهم عن نفسه ذلك بانهم لا يصيبهم ظما ولا نصب ولا مخمصه في سبيل الله ولا يطئون موطئا يغيظ الكفار ولا ينالون من عدو نيلا الا كتب لهم به عمل صالح']
Answer Start in Text:[347]
-

In [ ]:
# Dealing with long docs:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it

def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" ],
        examples["context" ],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    # Looks like [0,1,2,2,2,3,4,5,5...] - Here 2nd input pair has been split in 3 parts
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    # Looks like [[(0,0),(0,3),(3,4)...] ] - Contains the actual start indices and end indices for each word in the input.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    print("First===================")
    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
dataset.column_names

In [ ]:
model_checkpoint = "salti/AraElectra-base-finetuned-ARCD"

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
model_checkpoint = "salti/AraElectra-base-finetuned-ARCD"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)



In [ ]:
tokenized_datasets = dataset.map(prepare_train_features, batched=True, remove_columns=dataset.column_names)


In [ ]:
tokenized_datasets['input_ids'][0]

In [ ]:

args = TrainingArguments(
    f"test-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

data_collator = default_data_collator
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets,
#     eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()
trainer.save_model(trainer.save_model("test-squad-trained"))

In [43]:
dataset[0]

{'id': '9:60-61_316',
 'context': 'انما الصدقات للفقراء والمساكين والعاملين عليها والمؤلفه قلوبهم وفي الرقاب والغارمين وفي سبيل الله وابن السبيل فريضه من الله والله عليم حكيم . ومنهم الذين يؤذون النبي ويقولون هو اذن قل اذن خير لكم يؤمن بالله ويؤمن للمؤمنين ورحمه للذين امنوا منكم والذين يؤذون رسول الله لهم عذاب اليم . ',
 'question': 'ما هي مصارف الزكاه ؟ ',
 'answers': {'answer_start': [13],
  'text': ['للفقراء والمساكين والعاملين عليها والمؤلفه قلوبهم وفي الرقاب والغارمين وفي سبيل الله وابن السبيل']}}

In [46]:
dataset[0]['answers']

{'answer_start': [13],
 'text': ['للفقراء والمساكين والعاملين عليها والمؤلفه قلوبهم وفي الرقاب والغارمين وفي سبيل الله وابن السبيل']}

In [ ]:
dataset[0]['question']

In [56]:
# This is folder using our model
model_checkpoint = "models/model_4_torch"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
text = """كلا والقمر. والليل إذ أدبر. والصبح إذا أسفر. إنها لإحدى الكبر. نذيرا للبشر. لمن شاء منكم أن يتقدم أو يتأخر. كل نفس بما كسبت رهينة. إلا أصحاب اليمين. في جنات يتساءلون. عن المجرمين. ما سلككم في سقر. قالوا لم نك من المصلين. ولم نك نطعم المسكين. وكنا نخوض مع الخائضين. وكنا نكذب بيوم الدين. حتى أتانا اليقين. فما تنفعهم شفاعة الشافعين.
"""

questions = """ما هي الدلائل التي تشير بأن الانسان مخير؟"""

for question in questions:
    inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)

pred = model(**inputs)
answer_start_scores, answer_end_scores = pred['start_logits'][0] ,pred['end_logits'][0]

answer_start = torch.argmax(
    answer_start_scores
)  # Get the most likely beginning of answer with the argmax of the score
answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
print(f"Question: {question}")
print(f"Answer: {answer}\n")


Question: ؟
Answer: 



In [57]:
from transformers import pipeline

In [58]:
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)

pipe(question=questions, context=text, topk=10)


[{'score': 0.08747900277376175,
  'start': 76,
  'end': 106,
  'answer': 'لمن شاء منكم أن يتقدم أو يتأخر'},
 {'score': 0.05618338659405708,
  'start': 76,
  'end': 88,
  'answer': 'لمن شاء منكم'},
 {'score': 0.04262829199433327,
  'start': 63,
  'end': 106,
  'answer': 'نذيرا للبشر. لمن شاء منكم أن يتقدم أو يتأخر'},
 {'score': 0.040407273918390274,
  'start': 76,
  'end': 129,
  'answer': 'لمن شاء منكم أن يتقدم أو يتأخر. كل نفس بما كسبت رهينة'},
 {'score': 0.02737801894545555,
  'start': 63,
  'end': 88,
  'answer': 'نذيرا للبشر. لمن شاء منكم'},
 {'score': 0.01734713837504387, 'start': 76, 'end': 83, 'answer': 'لمن شاء'},
 {'score': 0.016793658956885338,
  'start': 74,
  'end': 106,
  'answer': '. لمن شاء منكم أن يتقدم أو يتأخر'},
 {'score': 0.015763359144330025,
  'start': 76,
  'end': 123,
  'answer': 'لمن شاء منكم أن يتقدم أو يتأخر. كل نفس بما كسبت'},
 {'score': 0.013342678546905518,
  'start': 63,
  'end': 74,
  'answer': 'نذيرا للبشر'},
 {'score': 0.012037534266710281,
  'start': 

In [13]:
test_passage_question_objects = read_JSONL_file('datasets/qrcd_v1.1_dev.jsonl')

Loaded 109 records from datasets/qrcd_v1.1_dev.jsonl
pq_id: 9:60-61_316
pq_id: 2:178-179_400
pq_id: 2:190-194_400
pq_id: 2:243-245_400
pq_id: 4:74-76_400
pq_id: 4:80-84_400
pq_id: 4:88-91_400
pq_id: 5:32-34_400
pq_id: 8:38-40_400
pq_id: 9:1-6_400
pq_id: 9:12-16_400
pq_id: 9:28-29_400
pq_id: 9:36-37_400
pq_id: 9:122-123_400
pq_id: 17:31-35_400
pq_id: 25:63-77_400
pq_id: 33:25-27_400
pq_id: 33:60-62_400
pq_id: 47:1-6_400
pq_id: 49:9-10_400
pq_id: 2:34-39_257
pq_id: 7:19-25_257
pq_id: 20:115-123_257
pq_id: 2:83-86_313
pq_id: 2:178-179_313
pq_id: 2:228-230_313
pq_id: 2:234-237_313
pq_id: 3:145-148_313
pq_id: 3:169-175_313
pq_id: 4:36-40_313
pq_id: 4:122-126_313
pq_id: 5:12-13_313
pq_id: 5:82-86_313
pq_id: 7:54-56_313
pq_id: 9:90-96_313
pq_id: 9:100-102_313
pq_id: 9:117-121_313
pq_id: 17:22-30_313
pq_id: 18:29-31_313
pq_id: 22:30-37_313
pq_id: 29:61-69_313
pq_id: 33:28-31_313
pq_id: 39:9-10_313
pq_id: 39:32-37_313
pq_id: 41:33-36_313
pq_id: 46:15-18_313
pq_id: 53:31-41_313
pq_id: 2:124-129_

In [14]:
test_df_file   = json_to_dataframe(test_passage_question_objects)